# Grover vs Classical Search Performance
*By Vivek Rajagopalan | Date: 24Dec2025 *

## Introduction
Searching for a target in an unsorted database is a common task. Classically, this takes **O(N)** time (check every element). Grover’s algorithm, a quantum search algorithm, achieves this in **O(√N)** queries—an exponential speedup!

This notebook compares Grover’s and classical search performance, including:
- **Time Complexity**: Theoretical and simulated query counts.
- **Noise Effects**: How real-world quantum noise impacts Grover’s success rate.

In [ ]:
# Install required libraries (uncomment and run if needed)
# !pip install qiskit matplotlib numpy

## Setup
Import libraries for quantum simulation, plotting, and data handling.

In [ ]:
import qiskit
from qiskit import Aer, execute
from qiskit.algorithms import Grover
from qiskit.algorithms.oracles import TruthTableOracle
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np
import random

## Problem Definition: Unsorted Search
We need to find a **target element** in an unsorted list of `N` unique elements. The goal is to minimize the number of queries (element checks).

- **Classical**: Check elements one by one (worst-case: `N` queries).
- **Grover’s**: Use quantum parallelism and amplitude amplification to find the target in ~`√N` queries.

In [ ]:
def classical_search_queries(N):
    unsorted_list = list(range(N))
    random.shuffle(unsorted_list)
    target_val = N - 1
    current_idx = unsorted_list.index(target_val)
    unsorted_list[current_idx], unsorted_list[-1] = unsorted_list[-1], unsorted_list[current_idx]
    for i in range(N):
        if unsorted_list[i] == target_val:
            return i + 1
    return N

N_example = 8
queries = classical_search_queries(N_example)
print(f"Classical search for N={N_example}: {queries} queries (worst-case)")

## Grover’s Algorithm Implementation
Grover’s algorithm uses a quantum circuit to amplify the amplitude of the target state.

**Steps**:
1. Oracle marks the target state.
2. Diffusion operator amplifies the amplitude.
3. Repeat ~π/4 * √N times.
4. Measure to reveal the target index.

In [ ]:
def grover_algorithm(N, target_index):
    n = N.bit_length()
    assert (1 << (n-1)) < N <= (1 << n), "N must be a power of 2"
    truth_table = ['0'] * N
    truth_table[target_index] = '1'
    truth_table = ''.join(truth_table)
    oracle = TruthTableOracle(truth_table)
    iterations = round(np.pi/4 * np.sqrt(N))
    grover = Grover(oracle=oracle, iterations=iterations)
    qc = grover.construct_circuit()
    qc.measure_all()
    simulator = Aer.get_backend('qasm_simulator')
    result = execute(qc, simulator, shots=1024).result()
    counts = result.get_counts()
    target_bitstring = bin(target_index)[2:].zfill(n)
    success_count = counts.get(target_bitstring, 0)
    success_prob = success_count / 1024
    return success_prob, iterations

success_prob, iterations = grover_algorithm(8, 5)
print(f"Grover's for N=8, target=5: Success probability={success_prob:.2f}, Iterations={iterations}")

## Time Complexity Comparison
Compare classical vs quantum query counts.

In [ ]:
N_values = [2, 4, 8, 16, 32, 64]
classical_queries = [classical_search_queries(N) for N in N_values]
quantum_iterations = [round(np.pi/4 * np.sqrt(N)) for N in N_values]
ideal_success_probs = [grover_algorithm(N, 0)[0] for N in N_values]

plt.figure(figsize=(10, 6))
plt.plot(N_values, classical_queries, '-o', label='Classical (O(N))')
plt.plot(N_values, quantum_iterations, '-s', label='Quantum (O(√N))')
plt.xlabel('Database Size (N)')
plt.ylabel('Number of Queries')
plt.title('Query Count Comparison: Grover vs Classical')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(N_values, ideal_success_probs, '-o', label='Grover (Ideal)')
plt.axhline(y=0.5, color='r', linestyle='--', label='Random Guess (0.5)')
plt.xlabel('Database Size (N)')
plt.ylabel('Success Probability')
plt.title('Grover’s Success Probability (Ideal)')
plt.legend()
plt.grid(True)
plt.ylim(0, 1)
plt.show()